In [3]:
import pandas as pd
import mysql.connector
import os

# List of CSV files and their corresponding table names
csv_files = [
    ('Car_Data.csv', 'car_data'),
    ('Owners_data.csv', 'owners_data'),
    ('Sales_data.csv', 'sales_data'),
    ('Service_History.csv', 'service_history'),
    ('Insurance_data.csv', 'insurance_data')
]

# Database connection parameters
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='Lenovo@4000',
    database='Car'
)
cursor = conn.cursor()

# Function to load CSV data into the database
folder_path = 'C:/Users/Abhijeet Kuanr/Downloads/Car Data'

def get_sql_type(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return 'INT'
    elif pd.api.types.is_float_dtype(dtype):
        return 'FLOAT'
    elif pd.api.types.is_bool_dtype(dtype):
        return 'BOOLEAN'
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return 'DATETIME'
    else:
        return 'TEXT'

for csv_file, table_name in csv_files:
    file_path = os.path.join(folder_path, csv_file)
    
    # Read the CSV file into a pandas DataFrame
    df = pd.read_csv(file_path)
    
    # Replace NaN with None to handle SQL NULL
    df = df.where(pd.notnull(df), None)
    
    # Debugging: Check for NaN values
    print(f"Processing {csv_file}")
    print(f"NaN values before replacement:\n{df.isnull().sum()}\n")

    # Clean column names
    df.columns = [col.replace(' ', '_').replace('-', '_').replace('.', '_') for col in df.columns]

    # Generate the CREATE TABLE statement with appropriate data types
    columns = ', '.join([f'`{col}` {get_sql_type(df[col].dtype)}' for col in df.columns])
    create_table_query = f'CREATE TABLE IF NOT EXISTS `{table_name}` ({columns})'
    cursor.execute(create_table_query)

    # Insert DataFrame data into the MySQL table
    for _, row in df.iterrows():
        # Convert row to tuple and handle NaN/None explicitly
        values = tuple(None if pd.isna(x) else x for x in row)
        sql = f"INSERT INTO `{table_name}` ({', '.join(['`' + col + '`' for col in df.columns])}) VALUES ({', '.join(['%s'] * len(row))})"
        cursor.execute(sql, values)

    # Commit the transaction for the current CSV file
    conn.commit()

# Close the connection
conn.close()


Processing Car_Data.csv
NaN values before replacement:
Car_ID          0
Brand           0
Model           0
Year            0
Fuel_Type       0
Transmission    0
Color           0
Owner_Type      0
Mileage_kmpl    0
Price_Lakh      0
dtype: int64

Processing Owners_data.csv
NaN values before replacement:
Car_ID           0
Owner_Name       0
Contact          0
City             0
Purchase_Year    0
dtype: int64

Processing Sales_data.csv
NaN values before replacement:
Car_ID             0
Sale_Price_Lakh    0
Sale_Date          0
Buyer_Name         0
dtype: int64

Processing Service_History.csv
NaN values before replacement:
Car_ID            0
Service_Type      0
Service_Date      0
Service_Cost      0
Service_Center    0
dtype: int64

Processing Insurance_data.csv
NaN values before replacement:
Car_ID           0
Provider         0
Policy_Number    0
Expiry_Date      0
Status           0
dtype: int64

